In [1]:
pip install tensorflow_datasets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_datasets as tfds
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load the Food101 dataset
(ds_train, ds_test), ds_info = tfds.load('food101', split=['train[:80%]', 'train[80%:]'], as_supervised=True, with_info=True)



In [3]:
# Define a function to resize images
def resize_image(image, label):
    image = tf.image.resize(image, (32, 32))
    return image, label

# Apply resizing to the dataset
ds_train = ds_train.map(resize_image)
ds_test = ds_test.map(resize_image)

# Normalize pixel values to be in the range [0, 1]
ds_train = ds_train.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
ds_test = ds_test.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))

# Shuffle and batch the datasets
ds_train = ds_train.shuffle(10000).batch(64)
ds_test = ds_test.batch(64)

# Convert labels to categorical format
def categorical_labels(images, labels):
    return images, tf.one_hot(labels, depth=101)  # 101 classes in Food101 dataset

ds_train = ds_train.map(categorical_labels)
ds_test = ds_test.map(categorical_labels)

# Verify the shapes
for images, labels in ds_train.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)

for images, labels in ds_test.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)



Images shape: (64, 32, 32, 3)
Labels shape: (64, 101)
Images shape: (64, 32, 32, 3)
Labels shape: (64, 101)


In [5]:

# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Add the top layers for classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(101, activation='softmax'))  # Change the number of units to 101 for Food101 dataset
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])



In [6]:
model.fit(ds_train, batch_size=64, epochs=50)

Epoch 1/50
947/947 [==============================] - 3072s 3s/step - loss: 4.8191 - accuracy: 0.0405
Epoch 2/50
947/947 [==============================] - 3901s 4s/step - loss: 4.4011 - accuracy: 0.0783
Epoch 3/50
947/947 [==============================] - 3971s 4s/step - loss: 3.8789 - accuracy: 0.1152
Epoch 4/50
947/947 [==============================] - 3919s 4s/step - loss: 3.6513 - accuracy: 0.1514
Epoch 5/50
947/947 [==============================] - 3877s 4s/step - loss: 3.5831 - accuracy: 0.1600
Epoch 6/50
947/947 [==============================] - 2775s 3s/step - loss: 3.4554 - accuracy: 0.1812
Epoch 7/50
947/947 [==============================] - 3783s 4s/step - loss: 3.3515 - accuracy: 0.2003
Epoch 8/50
947/947 [==============================] - 3982s 4s/step - loss: 3.2110 - accuracy: 0.2223
Epoch 9/50
947/947 [==============================] - 4023s 4s/step - loss: 3.0344 - accuracy: 0.2538
Epoch 10/50
947/947 [==============================] - 4007s 4s/step - loss: 2.875

In [7]:
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix

# Make predictions on the test dataset
predictions = model.predict(ds_test)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Manually generate ground truth labels for evaluation
true_labels = []
for _, label in ds_test:
    true_labels.extend(label.numpy())

true_labels = np.argmax(true_labels, axis=1)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Assuming you have test_loss and test_accuracy available
# If not, you'll need to calculate them based on your model and test dataset
print(f"Test Accuracy: {accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Test Accuracy: 0.17042904290429042
Accuracy: 0.17042904290429042
F1 Score: 0.1720990490234784
